## Queries - ComCam

In this notebook, we show how to query the ComCam repository\
and view the resulting images.\
This notebook uses the most recent calibrations
Craig Lage - 30-Aug-24

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable
import astropy.io.fits as pf
from lsst.daf.butler import Butler
from lsst.ip.isr import IsrTask, IsrTaskConfig
from lsst.summit.utils.plotting import plot
import lsst.afw.cameraGeom.utils as camGeomUtils

In [ ]:
butler = Butler('/repo/embargo_old', collections=["LSSTComCam/raw/all", "LSSTComCam/calib", "LSSTComCam/nightlyValidation"])
instrument = "LSSTComCam"

In [ ]:
butler = Butler('/sdf/group/rubin/repo/main', collections=["LSSTComCam/raw/all","LSSTComCam/calib"])
instrument = 'LSSTComCam'
expId = 2024112800374
raw = butler.get('raw', detector=3, exposure=expId, instrument='LSSTComCam')


#raw = butler.get('raw', detector=3, visit=expId, instrument='LSSTComCam')

## First, get a list of exposures
### These should match what you see in RubinTV.

In [ ]:
butler = Butler('/repo/embargo_new', collections=["LSSTComCam/raw/all", "LSSTComCam/calib", "LSSTComCam/nightlyValidation"])
dayObs = 20241119
instrument = "LSSTComCam"

exposureList = []
for record in butler.registry.queryDimensionRecords("exposure", 
                    where=f"exposure.day_obs={dayObs} and instrument='LSSTComCam'"):
    exposureList.append([record.id, record])
exposureList.sort(key=lambda x: x[0])
for [id,record] in exposureList:
    print(record.id, record.observation_type, record.exposure_time, record.physical_filter)


In [ ]:
record.seq_num

# Get the data from the headers

In [ ]:
butler = Butler('/repo/embargo_old', collections=["LSSTComCam/raw/all", "LSSTComCam/calib", "LSSTComCam/nightlyValidation"])
expId = 2024072300063
mData = butler.get('raw.metadata', detector=1, exposure=expId, instrument=instrument)
for key in mData.keys():
    print(key, mData[key])

# Now get the raw data

## Next, look at the raw data from one of the exposures.
### Because of the large pedestal, we don't see much.  We need to do ISR

In [ ]:
expId = 2024080800029
raw = butler.get('raw', detector=4, exposure=expId, instrument=instrument)


## Some people have expressed the desire to download the FITS files.  This cell will do that.  

In [ ]:
filename = "/home/c/cslage/u/ComCam/images/2024080800029_4_raw.fits"
raw.image.writeFits(filename)

In [ ]:
%matplotlib inline        
x = plot(raw, stretch='ccs')

## Define a simple ISR
### Just overscan subtraction and bias subtraction.

In [ ]:
isrConfig = IsrTaskConfig()
isrConfig.doLinearize=False
isrConfig.doOverscan=True
isrConfig.overscan.fitType="MEDIAN_PER_ROW"
isrConfig.overscan.doParallelOverscan=False
isrConfig.doAssembleCcd=True
isrConfig.doBias=False
isrConfig.doVariance=False
isrConfig.doCrosstalk=False
isrConfig.doBrighterFatter=False
isrConfig.doDark=False
isrConfig.doStrayLight=False
isrConfig.doFlat=False
isrConfig.doFringe=False
isrConfig.doApplyGains=False
isrConfig.usePtcGains=False
isrConfig.doDefect=False
isrConfig.doNanMasking=False
isrConfig.doInterpolate=False
isrConfig.doSaturation=False
isrConfig.doSaturationInterpolation=False
isrTask = IsrTask(config=isrConfig)

In [ ]:
expIds = [2024103000016, 2024103000052]
med_dict = {}
for expId in expIds:
    meds = []
    for detector in range(9):
        exp = butler.get('raw', detector=detector, exposure=expId, instrument=instrument)
        isrResult = isrTask.run(exp)
        med = np.nanmedian(isrResult.exposure.image.array)
        meds.append(med)
    med_dict[expId] = meds
    
    

In [ ]:
print(med_dict)

In [ ]:
x=plot(isrResult.exposure, stretch='ccs')

In [ ]:
expId = 2024111900027
exp = butler.get('postISRCCD', detector=3, exposure=expId, instrument=instrument)
#biasExp = butler.get('bias', detector=4, exposure=expId, instrument=instrument) # This is a bias image associated with the data
#flatExp = butler.get('flat', detector=4, exposure=expId, instrument=instrument) # This is a bias image associated with the data
#isrResult = isrTask.run(exp, bias=biasExp, flat=flatExp) # This runs the ISR
#isrResult = isrTask.run(exp) # This runs the ISR
x=plot(exp, stretch='ccs')
#plt.savefig(f"/home/c/cslage/u/ComCam/images/ComCam_Saturated_Raw_Full_2_{expId}.png")

In [ ]:
x = plot(exp.image.array[2500:3000, 3400:3500], stretch='ccs')
x.axes[0].set_title(f"ComCam {expId}, Det 8")
#x.axes[0].set_xlim(2500, 3000)
#x.axes[0].set_ylim(2000, 2090)
#plt.savefig(f"/home/c/cslage/u/ComCam/images/ComCam_Saturated_Raw_3_{expId}.png")

In [ ]:
Y1 = 3200
Y2 = 3400
plt.title(f"ComCam {expId}, Median of Y={Y1}-{Y2}")
plt.plot(np.nanmedian(exp.image.array[Y1:Y2, :], axis=0), marker='x')
plt.xlim(2750, 2850)
plt.ylim(30000, 31000)
plt.ylabel("Flux (electrons)")
plt.xlabel("Pixels")
plt.savefig(f"/home/c/cslage/u/ComCam/images/ComCam_Slice_2_{expId}.png")

In [ ]:
collections=['LSSTComCam/nightlyValidation']
registry = butler.registry
for dtype in registry.queryDatasetTypes()[:]:
    try:
        datasetRefs = list(registry.queryDatasets(datasetType=dtype,collections=collections))
        if len(datasetRefs) > 0:
            print(len(datasetRefs), dtype )
    except:
        print("Error", dtype )

In [ ]:
expId = 2024102800104
calexp = butler.get('calexp', detector=4, visit=expId)


In [ ]:
x=plot(calexp, stretch='ccs')
plt.savefig(f"/home/c/cslage/u/ComCam/images/ComCam_Calexp_{expId}.png")

In [ ]:
butler = Butler('/repo/embargo_new', collections=["LSSTComCam/raw/all", \
                "LSSTComCam/calib", "LSSTComCam/nightlyValidation", 'u/yusra/testPipelines'])
expId = 2024102700069
calExp = butler.get('calexp', detector=4, visit=expId, instrument=instrument)
rawExp = butler.get('raw', detector=4, exposure=expId, instrument=instrument)
cWcs = calExp.getWcs()
rWcs = rawExp.getWcs()
rawSkyCenter = rWcs.getSkyOrigin()
calExpSkyCenter = cWcs.pixelToSky(rWcs.getPixelOrigin())
deltaRa = rawSkyCenter.getRa().asArcseconds() - calExpSkyCenter.getRa().asArcseconds()
deltaDec = rawSkyCenter.getDec().asArcseconds() - calExpSkyCenter.getDec().asArcseconds()
print(f"DeltaRa = {deltaRa:.2f}, DeltaDec = {deltaDec:.2f}")

In [ ]:
instrument="LSSTComCam"
butler = Butler('/repo/embargo_new', collections=["LSSTComCam/raw/all", \
                "LSSTComCam/calib", "LSSTComCam/nightlyValidation"])
expId = 2024110600252
calExp = butler.get('calexp', detector=4, visit=expId, instrument=instrument)
rawExp = butler.get('raw', detector=4, exposure=expId, instrument=instrument)
cWcs = calExp.getWcs()
rWcs = rawExp.getWcs()
rawSkyCenter = rWcs.getSkyOrigin()
calExpSkyCenter = cWcs.pixelToSky(rWcs.getPixelOrigin())
deltaRa = rawSkyCenter.getRa().asArcseconds() - calExpSkyCenter.getRa().asArcseconds()
deltaDec = rawSkyCenter.getDec().asArcseconds() - calExpSkyCenter.getDec().asArcseconds()
print(f"DeltaRa = {deltaRa:.2f}, DeltaDec = {deltaDec:.2f}")

In [ ]:
expId = 2024110600252
exp = butler.get('postISRCCD', detector=4, exposure=expId, instrument=instrument)


In [ ]:
expId = 2024110600252
exp = butler.get('postISRCCD', detector=4, exposure=expId, instrument=instrument)
%matplotlib inline
x = plot(exp, stretch='ccs')
x.axes[0].set_title(f"ComCam {expId} Det 4", fontsize=18)
plt.savefig(f"/home/c/cslage/u/ComCam/images/ComCam_postISR_{expId}.png")

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(5,5))
im = ax.imshow(exp.image.array[2595:2615, 2008:2028], vmin=0, vmax=25000)
div = make_axes_locatable(ax)
cax = div.append_axes("right", size="5%", pad=0.05)
fig.colorbar(im, cax=cax)

ax.set_title(f"ComCam {expId}, Det 4")
plt.savefig(f"/home/c/cslage/u/ComCam/images/ComCam_Star_Core_{expId}.png")

In [ ]:
Y=2602
plt.title(f"ComCam {expId}, Y={Y}")
plt.plot(exp.image.array[Y, 1990:2040])
plt.axhline(14000, ls='--', color='black')
plt.axvline(24.8, ls='--', color='black')
plt.axvline(29.0, ls='--', color='black')
plt.text(32, 20000, "FWHM = 0.84 arcsec")
plt.ylim(0, 30000)
plt.ylabel("Flux (electrons)")
plt.xlabel("Pixels")
plt.savefig(f"/home/c/cslage/u/ComCam/images/ComCam_FWHM_Slice_{expId}.png")

In [ ]:
expId = 2024110600252
exp = butler.get('postISRCCD', detector=6, exposure=expId, instrument=instrument)
%matplotlib inline
x = plot(exp, stretch='ccs')
x.axes[0].set_title(f"ComCam {expId} Det 6", fontsize=18)
plt.savefig(f"/home/c/cslage/u/ComCam/images/ComCam_postISR_{expId}.png")

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(5,5))
im = ax.imshow(exp.image.array[2500:2520, 1685:1705], vmin=0, vmax=25000)
div = make_axes_locatable(ax)
cax = div.append_axes("right", size="5%", pad=0.05)
fig.colorbar(im, cax=cax)

ax.set_title(f"ComCam {expId}, Det 6")
plt.savefig(f"/home/c/cslage/u/ComCam/images/ComCam_Star_Core_{expId}.png")

In [ ]:
Y=2508
plt.title(f"ComCam {expId}, Y={Y}")
plt.plot(exp.image.array[Y, 1675:1725])
plt.axhline(12500, ls='--', color='black')
plt.axvline(19.0, ls='--', color='black')
plt.axvline(23.2, ls='--', color='black')
plt.text(32, 20000, "FWHM = 0.84 arcsec")
plt.ylim(0, 25000)
plt.ylabel("Flux (electrons)")
plt.xlabel("Pixels")
plt.savefig(f"/home/c/cslage/u/ComCam/images/ComCam_FWHM_Slice_{expId}.png")

In [ ]:
expId = 2024110400129
exp = butler.get('raw', detector=1, exposure=expId, instrument=instrument)
#biasExp = butler.get('bias', detector=4, exposure=expId, instrument=instrument) # This is a bias image associated with the data
#flatExp = butler.get('flat', detector=4, exposure=expId, instrument=instrument) # This is a bias image associated with the data
#isrResult = isrTask.run(exp, bias=biasExp, flat=flatExp) # This runs the ISR
isrResult = isrTask.run(exp) # This runs the ISR
x=plot(isrResult.exposure, stretch='ccs')


In [ ]:
x = plot(isrResult.exposure.image.array, stretch='ccs')
x.axes[0].set_title(f"ComCam {expId}, Det 1")
x.axes[0].set_xlim(1500, 1700)
x.axes[0].set_ylim(2500, 3200)
plt.savefig(f"/home/c/cslage/u/ComCam/images/ComCam_Sat_{expId}.png")

In [ ]:
Y = 2520
plt.title(f"ComCam {expId}, Y = {Y}")
plt.plot(exp.image.array[Y, :], marker='x')
plt.xlim(1900, 2000)
plt.axhline(3000, ls='--', color='k')
plt.axvline(1935, ls='--', color='k')
plt.axvline(1943.5, ls='--', color='k')
plt.text(1960, 5000, "FWHM = 1.7 arcseconds")
plt.ylabel("Flux (electrons)")
plt.xlabel("Pixels")
plt.savefig(f"/home/c/cslage/u/ComCam/images/ComCam_Slice_{expId}.png")

In [ ]:
import scipy.ndimage as ndimage
img = isrResult.exposure.image.array
#img = ndimage.gaussian_filter(img, sigma=(5,5), order=0)
x = plot(img, stretch='ccs')
x.axes[0].set_title(f"ComCam {expId}, Det 1")
x.axes[0].set_xlim(1000, 1300)
x.axes[0].set_ylim(2500, 3200)
plt.savefig(f"/home/c/cslage/u/ComCam/images/ComCam_Sat_{expId}.png")

In [ ]:
Y1 = 2400
Y2 = 2900
Y3 = 3500
img = isrResult.exposure.image.array
plt.subplots_adjust(hspace=0.5)
plt.suptitle(f"ComCam Saturated star{expId}")
plt.subplot(2,1,1)
plt.title(f"Y = {Y1}")
plt.plot(img[Y1, :], marker='x')
plt.xlim(1550, 1750)
plt.ylim(0, 5000)
plt.axvline(1610, ls='--', color='black')
plt.ylabel("Flux (electrons)")
plt.xlabel("Pixels")
plt.subplot(2,1,2)
plt.title(f"Y = median of {Y2} to {Y3}")
plt.plot(np.nanmedian(img[Y2:Y3, :], axis=0), marker='x')
plt.axvline(1610, ls='--', color='black')
plt.xlim(1550, 1750)
plt.ylim(600, 700)
#plt.subplot(3,1,3)
#plt.plot(np.nanmedian(img[Y2:Y3, :], axis=0), marker='x')
#plt.xlim(1550-509, 1750-509)
#plt.ylim(600, 700)

plt.savefig(f"/home/c/cslage/u/ComCam/images/ComCam_Slice_{expId}.png")

In [ ]:
expId = 2024110400129
exp = butler.get('raw', detector=1, exposure=expId, instrument=instrument)
%matplotlib inline
x = plot(exp, stretch='ccs')
x.axes[0].set_title(f"ComCam {expId} Det 4", fontsize=18)
plt.savefig(f"/home/c/cslage/u/ComCam/images/ComCam_Calexp_{expId}.png")

In [ ]:

x = plot(exp.image.array, stretch='ccs')
x.axes[0].set_title(f"ComCam {expId}, Det 1")
x.axes[0].set_xlim(1800, 2000)
x.axes[0].set_ylim(2250, 3500)
plt.savefig(f"/home/c/cslage/u/ComCam/images/ComCam_Sat_{expId}.png")

In [ ]:
Y = 3000
plt.title(f"ComCam {expId}, Y = {Y}")
plt.plot(exp.image.array[Y, :], marker='x')
plt.xlim(1800, 2000)
plt.ylim(23500, 24000)
plt.ylabel("Flux (electrons)")
plt.xlabel("Pixels")
plt.savefig(f"/home/c/cslage/u/ComCam/images/ComCam_Slice_{expId}.png")

In [ ]:
expIds = [2024102400077, 2024102400118, 2024102400202, \
          2024102500048, 2024102500091, 2024102500124]
medians = np.zeros([6,9])
for i, expId in enumerate(expIds):
    for det in range(9):
        exp = butler.get('postISRCCD', detector=det, exposure=expId, instrument=instrument)
        med = np.nanmedian(exp.image.array)
        medians[i, det] = med


In [ ]:
for det in range(9):
    ratios = []
    for i in range(6):
        ratios.append((medians[i,det] / medians[i,4]))
    print(f"Detector {det}, Mean ratio = {np.mean(ratios):.3f}, Std = {np.std(ratios):.3f}")

In [ ]:
x = plot(exp.image.array[1925:1975, 2100:2150], stretch='linear')
plt.savefig(f"/home/c/cslage/u/ComCam/images/ComCam_Star_Core_{expId}.png")

In [ ]:
Y=1943
plt.title(f"ComCam {expId}, Y={Y}")
plt.plot(exp.image.array[Y, 2100:2150])
plt.axhline(27500, ls='--', color='black')
plt.axvline(17.5, ls='--', color='black')
plt.axvline(25, ls='--', color='black')
plt.text(30, 40000, "FWHM = 1.5arcsec")
#plt.ylim(500, 1500)
plt.ylabel("Flux (electrons)")
plt.xlabel("Pixels")
plt.savefig(f"/home/c/cslage/u/ComCam/images/ComCam_FWHM_Slice_{expId}.png")

## Run the ISR and look at the result

In [ ]:
%matplotlib inline
x = plot(isrResult.exposure, stretch='ccs')
plt.savefig(f"/home/c/cslage/u/ComCam/images/ComCam_{expId}.png")

# Plot a small region

In [ ]:
x = plot(isrResult.exposure.image.array[2225:2275, 1350:1400])

# Get the value of a particular pixel

In [ ]:
isrResult.exposure.image.array[2100, 2100]

# Plot the cross-chip uniformity

In [ ]:
plt.title(f"ComCam {expId}, Bias")
plt.plot(isrResult.exposure.image.array[2100, :], label='X cut')
plt.plot(isrResult.exposure.image.array[:, 2100], label='Y cut')
plt.ylim(0,50000)
#plt.ylim(-50, 50)
plt.ylabel("Flux (electrons)")
plt.xlabel("Pixels")
plt.legend()
plt.savefig(f"/home/c/cslage/u/ComCam/images/ComCam_Slices_{expId}.png")

## Some people have expressed the desire to download the FITS files.  This cell will do that.  

In [ ]:
filename = "/home/c/cslage/u/ComCam/images/2024080800029_4_isr.fits"
isrResult.exposure.image.writeFits(filename)

# Now assemble all 9 CCDs and plot the result

In [ ]:
def myCallback(im, ccd, imageSource):
    """Assemble the CCD image and do basic ISR"""
    dayObs = imageSource.kwargs['day_obs']
    seqNum = imageSource.kwargs['seq_num']
    exp = imageSource.butler.get('raw', detector=ccd.getId(), day_obs=dayObs, seq_num=seqNum)
    biasExp = imageSource.butler.get('bias', detector=ccd.getId(), day_obs=dayObs, seq_num=seqNum)
    ptc = butler.get('ptc', detector=ccd.getId(), instrument=instrument,
                 collections="u/abrought/LSSTComCam/calibs/w_2024_28/ptc_r03.08132024a")
    isrResult = isrTask.run(exp, bias=biasExp, ptc=ptc) # This runs the ISR
    oim = isrResult.exposure.image
    return oim

In [ ]:
def myCallback(im, ccd, imageSource):
    dayObs = imageSource.kwargs['day_obs']
    seqNum = imageSource.kwargs['seq_num']
    exp = imageSource.butler.get('calexp', detector=ccd.getId(), day_obs=dayObs, seq_num=seqNum)
    print(np.nanmedian(exp.image.array))
    oim = exp.image
    return oim

In [ ]:
%matplotlib inline
instrument = "LSSTComCam"
camera = butler.get('camera', instrument=instrument)
fig = plt.figure(figsize=(12,12))
import lsst.afw.display as afwDisplay
disp = afwDisplay.Display(1, "matplotlib")
#disp.scale('linear', min = 45000, max=65000)
disp.scale('asinh', min=0, max=100)
dayObs = 20241028
seqNum = 104
dataType='calexp'
mos = camGeomUtils.showCamera(camera,
                              camGeomUtils.ButlerImage(butler, dataType, 
                                                       instrument=instrument, raft="R22",
                                                       day_obs=dayObs, seq_num=seqNum,
                                                       verbose=False, callback=myCallback,
                                                       background=np.nan),
                              binSize=1, display=disp, overlay=False,
                              title="%d %d" % (dayObs, seqNum))

plt.savefig(f"/home/c/cslage/u/ComCam/images/ComCam_CalExp_{dayObs}_{seqNum}.png")

# Plot the cross-detector uniformity

In [ ]:
%matplotlib inline
plt.title(f"ComCam {dayObs}_{seqNum}")
plt.plot(mos.array[6000, :], label="X slice")
plt.plot(mos.array[:, 6000], label="Y slice")
plt.ylim(0, 60000)
#plt.ylim(-50, 50)
plt.ylabel("Flux (electrons)")
plt.xlabel("Pixels")
plt.legend()
plt.savefig(f"/home/c/cslage/u/ComCam/images/ComCam_Slices_{dayObs}_{seqNum}.png")

# Plot a small region

In [ ]:
x = plot(mos.array[6000:6100, 6000:6100])

# The cell below will save the combined image as a FITS file

In [ ]:
filename = f"/home/c/cslage/u/ComCam/images/FITS_{dayObs}_{seqNum}.fits"
mos.writeFits(filename)

In [ ]:
type(raw.image)

# Variability per amp

In [ ]:
%matplotlib inline
clip = 10.0
expId = 2024071200007

for det in range(9):
    exp = butler.get('raw', detector=det, exposure=expId)
    biasExp = butler.get('bias', detector=det, exposure=expId) # This is a bias image associated with the data
    isrResult = isrTask.run(exp, bias=biasExp) # This runs the ISR
    for amp in exp.getDetector().getAmplifiers():
        data = isrResult.exposure[amp.getBBox()].image.array.flatten()
        data = (data - np.median(data)) / np.median(data) * 100.0
        sortedData = np.sort(data)
        sortedTrimmedData = np.clip(sortedData, np.percentile(sortedData, clip), np.percentile(sortedData, 100 - clip))
        plt.plot(sortedTrimmedData)
plt.ylim(-5.0, 5.0)
plt.title(f"ComCam {expId}, per amp variability - 10->90 percentiles")
plt.xlabel("Pixel count")
plt.ylabel("Percent variation from Median")
plt.savefig(f"/home/c/cslage/u/ComCam/images/ComCam_Amp_Nonuniformity_{expId}.png")

In [ ]:
disp.

In [ ]:
butlerEmbargo = Butler('embargo_new', collections=['LSSTComCam/nightlyValidation'])
dataRefs = list(butlerEmbargo.registry.queryDatasets('calexp', where=f"instrument='LSSTComCam' \
and day_obs = 20241102").expanded())

In [ ]:
len(dataRefs)